In [1]:
import pandas as pd
import glob
import os

data_v = 'VersionD'

# 定义路径
slp_path = f'/shared5/RESEARCH_DATA/SWOT/{data_v}/processed'

# 读取 SWOT 数据并获取唯一的 reach_id
swot_df = pd.read_csv(f'1.all_matched_points_{data_v}.csv')
unique_reach_ids = swot_df['reach_id'].unique()

# 获取所有 CSV 文件
slp_csv_files = glob.glob(os.path.join(slp_path, 'reach*.csv'))

# 初始化一个空列表来存储匹配的数据
matched_data = []

# 遍历每个 CSV 文件
for csv_file in slp_csv_files:
    temp_df = pd.read_csv(csv_file)
    filtered_df = temp_df[temp_df['reach_id'].isin(unique_reach_ids)]
    matched_data.append(filtered_df)

# 合并所有匹配的数据
if matched_data:
    matched_df = pd.concat(matched_data, ignore_index=True)
    slp_df = matched_df[matched_df['slope']>0.0001]
    slp_df = slp_df[slp_df['slope_r_u']<10**(-4.0)]
    slp_df = slp_df[slp_df['reach_q'] <= 1]  # quality == good (0) or suspect (1)
    slp_df = slp_df[slp_df['reach_q_b'] & (1 << 1) == 0]   # big_slope_unc == 0（不确定度/坡度 ≤0.4）
    slp_df = slp_df[slp_df['reach_q_b'] & (1 << 13) == 0]  # far_range_suspect == False
    slp_df = slp_df[slp_df['reach_q_b'] & (1 << 14) == 0]  # near_range_suspect == False
    slp_df = slp_df[slp_df['dark_frac'] <= 0.5]   # dark water fraction
    slp_df = slp_df[slp_df['ice_dyn_f'] <= 1]     # ice-free or partially covered by ice
    slp_df = slp_df[slp_df['obs_frac_n'] >= 0.5]  # fraction of nodes with valid WSE
    slp_df = slp_df[slp_df['xovr_cal_q'] <= 1]    # crossover calibration quality indicator

    
    # 添加 date 列
    slp_df['date'] = pd.to_datetime('2000-01-01') + pd.to_timedelta(slp_df['time'], unit='s')
    slp_df['date'] = slp_df['date'].dt.date
    
    # 输出到 CSV
    output_file = f'3.swot_slope_{data_v}.csv'
    slp_df.to_csv(output_file, index=False)
    counts = slp_df.groupby('reach_id').size().reset_index(name='count')
    print(len(counts))
    counts = counts[counts['count']>6]
    print(len(counts))
    
    
    print(f"\n成功!")
    print(f"总共匹配了 {len(slp_df)} 条记录")
    print(f"包含 {slp_df['reach_id'].nunique()} 个唯一的 reach_id")
    print(f"数据已保存到: {output_file}")
    print(f"\n数据预览:")
    print(slp_df.head())
else:
    print("没有找到匹配的数据")

543
201

成功!
总共匹配了 3906 条记录
包含 543 个唯一的 reach_id
数据已保存到: 3.swot_slope_VersionD.csv

数据预览:
       reach_id          time      p_lat       p_lon       wse    wse_u  \
12  81260600011  8.049968e+08  67.597766 -139.738593  243.7939  0.09348   
47  72558200191  8.050642e+08  44.623218  -83.846507  255.8253  0.10309   
56  73286000021  8.050640e+08  32.447932  -86.191221   40.8413  0.10024   
82  82237000091  8.050222e+08  62.488943 -123.448330  108.6780  0.09328   
84  82289800011  8.050224e+08  54.005707 -116.909855  847.3062  0.09478   

         width   width_u  reach_q  reach_q_b  ...    slope2  slope2_r_u  \
12  106.449722  0.631234        1         12  ...  0.000542    0.000002   
47   67.311614  0.434090        1          4  ...  0.000957    0.000008   
56   85.863829  0.607932        0          0  ...  0.000143    0.000002   
82  214.133752  0.961828        1         12  ...  0.000595    0.000002   
84   86.306677  0.666307        1          4  ...  0.003328    0.000001   

    xovr